In [1]:
!pip install langchain
!pip install langchain-community
!pip install pymupdf
!pip install faiss-cpu
!pip install transformers
!pip install sentence-transformers

In [2]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import HuggingFacePipeline
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer

# 단계 1: 문서 로드(Load Documents)
loader = PyMuPDFLoader("SPRi AI Brief 5월호 산업동향.pdf")
docs = loader.load()

# 단계 2: 문서 분할(Split Documents)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
split_documents = text_splitter.split_documents(docs)

# 단계 3: 임베딩(Embedding) 생성
embeddings = HuggingFaceEmbeddings(
        model_name="BM-K/KoSimCSE-roberta-multitask"
        # model_name = 'jhgan/ko-sroberta-nli'
    )

# 단계 4: DB 생성(Create DB) 및 저장
# 벡터스토어를 생성합니다.
vectorstore = FAISS.from_documents(documents=split_documents, embedding=embeddings)

# 단계 5: 검색기(Retriever) 생성
# 문서에 포함되어 있는 정보를 검색하고 생성합니다.
retriever = vectorstore.as_retriever()

# 단계 6: 프롬프트 생성(Create Prompt)
# 프롬프트를 생성합니다.
prompt = PromptTemplate.from_template(
    """다음 컨텍스트를 사용하여 질문에 답변하세요.
    답변은 간결하고 정확해야 하며, 한국어로 작성하세요.
    컨텍스트에 없는 정보는 사용하지 마세요.

#Context:
{context}

#Question:
{question}

#Answer:"""
)

# 단계 7: 언어모델(LLM) 생성
# 모델(LLM) 을 생성합니다.
model_name = "beomi/Llama-3-Open-Ko-8B-Instruct-preview"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

llm_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    device=0,  # CPU 사용, GPU 사용 시 0 또는 적절한 디바이스 ID
)
llm = HuggingFacePipeline(pipeline=llm_pipeline)

# 단계 8: 체인(Chain) 생성
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

<ipython-input-2-cf83c8cba5ee>:20: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/764 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/442M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/752k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Device set to use cuda:0
<ipython-input-2-cf83c8cba5ee>:61: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=llm_pipeline)


In [3]:
# 체인 실행(Run Chain)
# 문서에 대한 질의를 입력하고, 답변을 출력합니다.
question = "EU집행위원회는 AI에 얼마를 투자했지?"
response = chain.invoke(question)
print(response)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


다음 컨텍스트를 사용하여 질문에 답변하세요. 
    답변은 간결하고 정확해야 하며, 한국어로 작성하세요. 
    컨텍스트에 없는 정보는 사용하지 마세요.

#Context: 
[Document(id='e0084cbe-729f-42e9-8e80-b003f511602e', metadata={'producer': 'Hancom PDF 1.3.0.505', 'creator': 'Hancom PDF 1.3.0.505', 'creationdate': '2025-05-09T09:07:04+09:00', 'source': 'SPRi AI Brief 5월호 산업동향.pdf', 'file_path': 'SPRi AI Brief 5월호 산업동향.pdf', 'total_pages': 28, 'format': 'PDF 1.4', 'title': '', 'author': 'dj', 'subject': '', 'keywords': '', 'moddate': '2025-05-09T09:07:04+09:00', 'trapped': '', 'modDate': "D:20250509090704+09'00'", 'creationDate': "D:20250509090704+09'00'", 'page': 4}, page_content='정책･법제\n기업･산업\n기술･연구\n인력･교육\n3\nEU 집행위원회, EU의 AI 리더십을 위한 AI 대륙 행동계획 발표\nn EU 집행위원회가 EU의 AI 리더십을 확보하기 위한 ‘AI 대륙 행동계획’을 수립하고 \n‘InvestAI’ 이니셔티브를 통해 AI에 총 2,000억 유로를 투자할 계획\nn AI 대륙 행동계획은 △AI 컴퓨팅 인프라 구축 △데이터 접근성 확대 △전략적 영역의 AI 촉진 \n△AI 역량과 인재 육성 △AI 법 시행 간소화의 5개 영역을 중점 추진\nKEY Contents\n£ AI 대륙 행동계획, 컴퓨팅 인프라와 데이터 접근성 확보 및 AI 도입 확대를 추진\nn EU 집행위원회가 2025년 4월 9일 EU의 AI 리더십을 확보하기 위

In [4]:
question = "구글이 발표한 에이전트 간 상호운용성을 보장하기 위한 개방형 통신 프로토콜은?"
response = chain.invoke(question)
print(response)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


다음 컨텍스트를 사용하여 질문에 답변하세요. 
    답변은 간결하고 정확해야 하며, 한국어로 작성하세요. 
    컨텍스트에 없는 정보는 사용하지 마세요.

#Context: 
[Document(id='b1bfc58a-28c8-4090-a7c0-98f9c3809918', metadata={'producer': 'Hancom PDF 1.3.0.505', 'creator': 'Hancom PDF 1.3.0.505', 'creationdate': '2025-05-09T09:07:04+09:00', 'source': 'SPRi AI Brief 5월호 산업동향.pdf', 'file_path': 'SPRi AI Brief 5월호 산업동향.pdf', 'total_pages': 28, 'format': 'PDF 1.4', 'title': '', 'author': 'dj', 'subject': '', 'keywords': '', 'moddate': '2025-05-09T09:07:04+09:00', 'trapped': '', 'modDate': "D:20250509090704+09'00'", 'creationDate': "D:20250509090704+09'00'", 'page': 10}, page_content='정책･법제\n기업･산업\n기술･연구\n인력･교육\n9\n구글, AI 에이전트 간 통신 프로토콜 ‘A2A’ 공개 및 MCP 지원 발표\nn 구글이 에이전트 간 상호운용성을 보장하기 위한 개방형 통신 프로토콜 A2A를 공개했으며, \nA2A는 에이전트 간 기능 탐색, 작업 관리, 협업, 사용자 경험 협의 등의 다양한 기능을 지원\nn 구글은 제미나이 모델과 SDK에서 앤스로픽의 MCP 지원을 추가하기로 했으며, A2A가 MCP보다 \n상위 계층의 프로토콜로서 MCP를 보완한다고 설명\nKEY Contents\n£ A2A, 다중 에이전트 간 협업을 위한 개방형 프로토콜로 설계\nn 구글(Google)이 2025년 4월 9일 50개 이상의 기업*과 협력해 AI 에이전트 